In [2]:
from selenium import webdriver
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests

In [15]:
driver = webdriver.Chrome()
g_driver = webdriver.Chrome()
g_driver.get("http://google.com")

In [17]:
def check_google(mid_first_name, last_name, school_url):
    faculty = False
    for url in school_url:
        if faculty == True:
            break
        g_driver.get("http://google.com")
        search = g_driver.find_element_by_name('q')
        if not ' ' in mid_first_name:
            search.send_keys(mid_first_name + ' ' + last_name + ' ' + url)
        else: 
            search.send_keys(mid_first_name + ' ' + last_name + ' ')
        search.send_keys(Keys.RETURN)
        elems = g_driver.find_elements_by_xpath("//a[@href]")

        for elem in elems:
            if url in elem.text:
                print(url + ' in: ' + elem.text)
                faculty = True
                break
    return faculty

In [81]:
def split_name(full_name):
    space_count = full_name.count(' ')
    if space_count == 1:
        f_name = full_name.split(' ')[0] 
        last_name = full_name.split(' ')[1]
    elif space_count == 2:
        f_name = full_name.split(' ')[0] 
#                         m_name = full_name.split(' ')[1]
#         fm_name = f_name + ' ' + m_name
        last_name = full_name.split(' ')[2]
    elif space_count > 2:
        f_name = full_name.split(' ')[0] 
#                         m_name = full_name.split(' ')[1]
#         fm_name = f_name + ' ' + m_name
        last_name = ' '.join(full_name.split(' ')[2:])
    return f_name, last_name

In [88]:
def get_links(first_name, last_name, school_url, school_name):
    driver.get("https://scholar.google.com/citations?view_op=search_authors&mauthors=" +  first_name + '+' + last_name + "&hl=en&oi=ao")
    i = 1
    try:
        print(i)
        element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
    except:
        element = []
    while element:
        full_name = element.text
        print(full_name)
        f_name, last_name = split_name(full_name)
        if last_name in full_name and f_name in full_name:
            affiliation = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/div[1]')
            print(affiliation.text.lower())
            if (school_name in affiliation.text.lower() or school_url in affiliation.text.lower()) and full_name not in correct_names:
                link_elm = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3/a')
                correct_name = (full_name, link_elm.get_attribute('href'))
                return correct_name
#             else:
#                 faculty = check_google(f_name, last_name, [school_url])
#                 if faculty and full_name not in correct_names:
#                     link_elm = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3/a')
#                     correct_name = (full_name, link_elm.get_attribute('href'))
#                     return correct_name
        i += 1
        try:
            element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
        except:
            element = []


In [78]:
# element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3')
element.text

'Kenneth W Abbott'

In [68]:
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

In [94]:
os.getcwd()
path = os.path.join(os.getcwd(), 'school_data')
files = os.listdir(path)
school_name = 'arizona'
school_url = 'asu'
for file in files[0:1]:
    main_df = pd.DataFrame(columns = ['Name', 'School', 'Link'])
    data = pd.read_csv(os.path.join(os.getcwd(), 'school_data', file))
    for i in range(len(data)):
        got_page = False
        mid_first_name = data['First Name'][i]
        last_name = data['Last Name'][i]
        full_name = mid_first_name + ' ' +  last_name
        school = data['School'][i]
        link = get_links(mid_first_name, last_name, school_url, school_name)
        print(link)
        driver.get('http://amazon.com')
        time.sleep(3)
        driver.get('http://facebook.com')
        time.sleep(3)
        if link:
            main_df = main_df.append(pd.DataFrame({'Name': [full_name], 'School': [school],'Link': [link[1]]}))
        else:
            main_df = main_df.append(pd.DataFrame({'Name': [full_name], 'School': [school],'Link': ['NA']}))

1
Kenneth W Abbott
jack e brown chair in law emeritus, professor of global studies, arizona state university
('Kenneth W Abbott', 'https://scholar.google.com/citations?hl=en&user=hva7aGkAAAAJ')
1
Angela D Banks
university of san francisco
Angela Abbate
economist, swiss national bank
None
1
None
1
Daniel Bodansky
arizona state university
('Daniel Bodansky', 'https://scholar.google.com/citations?hl=en&user=bqmEUsQAAAAJ')
1
Diana M Bowman
arizona state university
('Diana M Bowman', 'https://scholar.google.com/citations?hl=en&user=wSvWb9MAAAAJ')
1
Karen Lee Bradshaw
associate professor, rhodes university
Karen Bradshaw
arizona state university, sandra day o'connor school of law
('Karen Bradshaw', 'https://scholar.google.com/citations?hl=en&user=WzTErjUAAAAJ')
1
None
1
Andrew Carter
professor of earth sciences, birkbeck, university of london
Elizabeth J. Carter
carnegie mellon university
Dorinda Carter Andrews
associate dean for equity and inclusion and associate professor of teacher educat

In [95]:
main_df

,Name,School,Link
0,Kenneth W. Abbott,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Angela Banks,Arizona State University Sandra Day O'Connor C...,NA
0,Paul Bender,Arizona State University Sandra Day O'Connor C...,NA
0,Daniel Bodansky,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Diana Bowman,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Karen Bradshaw,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Charles Calleros,Arizona State University Sandra Day O'Connor C...,NA
0,Andrew Carter,Arizona State University Sandra Day O'Connor C...,NA
0,Susan Chesler,Arizona State University Sandra Day O'Connor C...,NA
0,Adam Chodorow,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
